In [1]:
FAMAGA_DOWNLOAD_HTML_URL = "https://test-api.famaga.org/imap"
FAMAGA_DOWNLOAD_HTML_TOKEN = "YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw"
EXTRACTED_DEALS_DATABASE_PATH = 'extracted_deals_messaging.db'
CLIENTS_HISTORY_PATH = r'C:\Users\MGroup\Documents\products.json'
GPT_DB_LOGGER_PATH = 'sqlite:///prompt_versions.db'

AGENTS_API_URL = 'http://localhost:8000'

In [2]:
from importlib import reload
import clients

reload(clients)
reload(clients.msg_html_extracter)

from clients.agents import AgentsAPIClient
from clients.client_statistics import ClientStatisticsService
from clients.clients_deals_history import ClientDealsHistoryRepository
from clients.email_downloader import EmailDownloader
from clients.extracted_deals import ExtractedDealsRepository
from clients.gpt_database_logger import GPTDatabaseLogger
from clients.msg_html_extracter import MsgHtmlExtracter

print('Updated')

AttributeError: module 'clients' has no attribute 'msg_html_extracter'

In [144]:
deals_rep = ExtractedDealsRepository(EXTRACTED_DEALS_DATABASE_PATH)
client_deals_rep = ClientDealsHistoryRepository(CLIENTS_HISTORY_PATH)
client_statistics = ClientStatisticsService()
downloader = EmailDownloader(FAMAGA_DOWNLOAD_HTML_URL, FAMAGA_DOWNLOAD_HTML_TOKEN)
client = AgentsAPIClient(AGENTS_API_URL)
db_logger = GPTDatabaseLogger(GPT_DB_LOGGER_PATH)

In [11]:
import json
import re

def select_json_block(text: str):
    match = re.search(r"```json\n([\s\S]*?)\n```", text)
    if match:
        json_data = match.group(1)
    else:
        raise ValueError("No valid JSON data found in the string.")

    return json.loads(json_data)

In [128]:
from typing import List, Literal, Optional
from pydantic import BaseModel, Field
import json


class Intent(BaseModel):
    intent: str
    sub_intent: str
    branch: str


class Message(BaseModel):
    id: str  # Adjust the type as needed (e.g., int for integer IDs)
    body: str
    sign: Optional[str] = None
    from_: Literal['customer', 'manager'] = Field(..., alias='from')
    intents: List[Intent]

    class Config:
        allow_population_by_field_name = True

In [109]:
msg = """
Dear Sir or Madam,
Please advise if stators and rotors are available for the attached. The pump appears to be very similar to a Mono pump
Kind regards
Austen Brunton BEng (Hons), MIMarEST
Technical Fleet Manager
WYM Group Ltd
Incorporating Wilson Yacht Management and Wilson Ship Management
New Email Address: austen@wymgroup.com
Web: www.wymgroup.com
Phone: +44 (0) 1482 648322
Mob: +44 (0) 7760665835
"""

### Get message intents

In [110]:

def get_message_intents(msg: str):
    response = db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Please attach to each message in chat history tags, annotations, indexes, intents to better classification and search.
The message could have one or more intents.

Message:
```
{msg}
```

For example, this message would have intents:
- customer: Hello So as not to waste precious time. We order devices from offer no. 440822 of friday, 29 september 2023 . 
If possible, please provide an additional discount on your purchase. Please confirm acceptance of our order. 
In the meantime, once you know what the transport cost will be, we will discuss how to deliver the parcel to us 
or whether we will collect it ourselves.

Example of intents/sub-intents/branches namings:
   - Order Processing -> Order Placement Confirmation -> Confirmation of Specific Offer
   - Order Processing -> Order Acceptance Confirmation -> Confirmation of Order Receipt and Acceptance
   - Pricing and Quotes -> Discount Inquiry -> Request for Additional Discount on Purchase
   - Pricing and Quotes -> Transport Cost Inquiry -> Inquiry About Delivery Costs
   - Delivery and Shipping -> Delivery Method Discussion -> Discussing Whether to Ship or Self-Collect
   - Delivery and Shipping -> Discussing Logistics -> Coordination of Transport and Delivery Options
   - Product Inquiry -> Availability Check

Put the result at ```json``` format, like:
```json
[
    {{
        "id": "<message_id>",
        "body": "<message body>", //only body without sign
        "sign": "<signature>", //message signature
        "from": "<message from>", //this should be one of two values: customer or manager
        "intents": [
            {{
                "intent": "<main intent>",
                "sub_intent": "<sub intent>",
                "branch": "<branch>"
            }}
        ]
    
    }}
]
```
Please put new lines symbols if it suitable for context \\n at body and sign fields.
    """}                             
    ])
    response_raw_json = select_json_block(response)
    parsed_messages = [Message(**message) for message in response_raw_json]
    return parsed_messages

### Get message details

In [112]:
def get_details_from_message(msg):
    response = db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Try to extract from text brand name, amount, detail name, part number from the text. Also recognize country by text.
<<<>>>
Dear Sir or Madam,\nPlease advise if stators and rotors are available for the attached. 
The pump appears to be very similar to a Mono pump\nKind regards
<<<>>>

If you cannot recognize specified parameters please put `null` value.

---

Your response should be a list of comma separated values, eg: `foo, bar, baz`

The output should be a markdown code snippet formatted in the following adr, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{{
    "parts": [
        {{
           "amount": int // This is the amount of details
           "brand_name": string // This is thr brand  name of detail
           "part_number": string // This is the part number of detail
        }}
    ],
    "client": {{
        "country": string // This is the country of detail,
        "domain": string // customer company domain
        "email": string // customer email,
        "office_country": string // country of customer office
    }}
}}
```
    """}                             
    ])
    return response

<>:35: SyntaxWarning: invalid escape sequence '\`'
<>:35: SyntaxWarning: invalid escape sequence '\`'
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28008\2480512955.py:35: SyntaxWarning: invalid escape sequence '\`'
  """}


In [ ]:
parsed_messages = get_message_intents(msg)

In [ ]:
msg_details = get_details_from_message(msg)

### Build SQL query

In [122]:
def build_sql_query(msg_details, intents):
    response = db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Here is the table.

TABLE product_inquiries (
    intent TEXT,
    sub_intent TEXT,
    is_brand_name BOOLEAN,
    is_part_number BOOLEAN,
    action TEXT
)

Please build sql query that select one or more rows by presented data. Please ignore action field at sql request condition. Database: sqlite

{intents}

Message details:
{msg_details}
---

Put the result at ```json``` format, like:
```sql
SELECT * FROM table_name
WHERE field1 = 'value_2'
```
    """}                             
    ])
    match = re.search(r"```sql\n([\s\S]*?)\n```", response)
    if match:
        return match.group(1)
    else:
        raise ValueError("No valid JSON data found in the string.")

    return json.loads(json_data)
    return response_raw_json

In [124]:
intents_str = '\n'.join([f'{intent.intent} -> {intent.sub_intent}' for intent in parsed_messages[0].intents])
intents_str
raw_sql = build_sql_query(msg_details, intents_str)

Based on the provided JSON data, the SQL query to select one or more rows from the `product_inquiries` table would be:

```sql
SELECT * FROM product_inquiries
WHERE intent = 'Product Inquiry' 
AND sub_intent = 'Availability Check' 
AND is_brand_name = 1 
AND is_part_number = 0
```

This query will select all rows where the `intent` is 'Product Inquiry', the `sub_intent` is 'Availability Check', the `is_brand_name` is true (1), and the `is_part_number` is false (0).

Please note that in SQLite, boolean values are represented as integers, where 1 is true and 0 is false.

--------------------

Note saved without feedback. ID: 236a0544-1e50-4a83-9630-03360b661e98
Input tokens: 177 Output tokens: 145 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [45]:
for row in rows:
    action = row[4]
    print(action)

ask_specify_part_number


### Ask specify part number

In [52]:
response = db_logger.create_completion(temperature=0.3, tags='intent_classifier', model='gpt-4', messages=[
{ "role": "user", "content": f"""
You are sales manager that sale parts and components for manufacturers.
As a Senior sales manager, please ask customer to specify part number for part. When you build your own message please read the examples of 
other sales managers [EXAMPLE MESSAGES].

Metadata:
{{
    "parts": [
        {{
           "amount": null,
           "brand_name": "Mono",
           "part_number": null
        }}
    ],
    "client": {{
        "country": null,
        "domain": null,
        "email": null,
        "office_country": null
    }}
}}

The example of previous messages from sales use it to build your own:

[EXAMPLE MESSAGES]
Message 1: ```
Dear Sir / Madam,
Thank you for your request!
Could you please tell us the article number?
```
[/EXAMPLE MESSAGES]
"""}                             
])

Dear Valued Customer,

Thank you for your interest in our Mono brand parts. In order to assist you more efficiently, could you please provide us with the specific part number you are interested in?

Looking forward to your response.

--------------------

Note saved without feedback. ID: 13bafa8b-f97e-481b-b50e-6ed5a9d37a4c
Input tokens: 169 Output tokens: 45 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

### Google sheet reader

In [81]:
import sqlite3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pydantic import BaseModel, Field, field_validator


class ProductInquiry(BaseModel):
    intent: str
    sub_intent: str
    is_brand_name: bool
    is_part_number: bool
    action: str

    @field_validator('is_brand_name', 'is_part_number')
    def convert_int_to_bool(cls, v):
        if isinstance(v, bool):
            return v  # In case the input is already a boolean
        if v in (1, 'TRUE', 'True', 'true'):
            return True
        elif v in (0, 'FALSE', 'False', 'false'):
            return False
        raise ValueError('Must be a boolean or equivalent integer (0 or 1)')



class GoogleSheetToSQLiteConverter:
    def __init__(self, spreadsheet: str, credentials_path: str):
        scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        self.spreadsheet = spreadsheet
        self.client = gspread.authorize(creds)
        
    def get_sheet_data(self, sheet: str):
        worksheet = client.open(self.spreadsheet).worksheet(sheet)
        records = worksheet.get_all_records()

        return records

### Init sql db

In [101]:
import sqlite3


def init_sqlite_db(conn, product_inquiries):
    cursor = conn.cursor()
    
    product_inquiries_data = [
        [
            inquiry.intent,
            inquiry.sub_intent,
            int(inquiry.is_brand_name),  # Convert boolean to int
            int(inquiry.is_part_number),  # Convert boolean to int
            inquiry.action
        ]
        for inquiry in product_inquiries
    ]

    print()
    cursor.execute('''CREATE TABLE product_inquiries (
        intent TEXT,
        sub_intent TEXT,
        is_brand_name BOOLEAN,
        is_part_number BOOLEAN,
        action TEXT
    )''')
    
    cursor.executemany('INSERT INTO product_inquiries VALUES (?, ?, ?, ?, ?)', product_inquiries_data)
    conn.commit()
    print('Data was added to db successfully.')

## Test

In [ ]:
msg = """
Dear Sir or Madam,
Please advise if stators and rotors are available for the attached. The pump appears to be very similar to a Mono pump
Kind regards
Austen Brunton BEng (Hons), MIMarEST
Technical Fleet Manager
WYM Group Ltd
Incorporating Wilson Yacht Management and Wilson Ship Management
New Email Address: austen@wymgroup.com
Web: www.wymgroup.com
Phone: +44 (0) 1482 648322
Mob: +44 (0) 7760665835
"""

In [130]:
parsed_messages = get_message_intents(msg)

```json
[
    {
        "id": "1",
        "body": "Dear Sir or Madam,\nPlease advise if stators and rotors are available for the attached. The pump appears to be very similar to a Mono pump\nKind regards",
        "sign": "Austen Brunton BEng (Hons), MIMarEST\nTechnical Fleet Manager\nWYM Group Ltd\nIncorporating Wilson Yacht Management and Wilson Ship Management\nNew Email Address: austen@wymgroup.com\nWeb: www.wymgroup.com\nPhone: +44 (0) 1482 648322\nMob: +44 (0) 7760665835",
        "from": "customer",
        "intents": [
            {
                "intent": "Product Inquiry",
                "sub_intent": "Availability Check",
                "branch": "Inquiry About Availability of Specific Parts"
            }
        ]
    
    }
]
```

--------------------

Note saved without feedback. ID: 97aafac4-d1df-4feb-b2c7-317e62fe7734
Input tokens: 504 Output tokens: 198 Total price: 0.03$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [131]:
msg_details = get_details_from_message(msg)

```json
{
    "parts": [
        {
           "amount": null,
           "brand_name": "Mono",
           "part_number": null
        }
    ],
    "client": {
        "country": null,
        "domain": null,
        "email": null,
        "office_country": null
    }
}
```

--------------------

Note saved without feedback. ID: 902181af-9f31-4a84-8842-ac887f9fbe1f
Input tokens: 247 Output tokens: 70 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [132]:
intents_str = '\n'.join([f'{intent.intent} -> {intent.sub_intent}' for intent in parsed_messages[0].intents])
raw_sql = build_sql_query(msg_details, intents_str)

Based on the provided JSON data, the SQL query would look like this:

```sql
SELECT * FROM product_inquiries
WHERE intent = 'Product Inquiry' 
AND sub_intent = 'Availability Check' 
AND is_brand_name = 1 
AND is_part_number = 0
```

This query will select all rows from the `product_inquiries` table where `intent` is 'Product Inquiry', `sub_intent` is 'Availability Check', `is_brand_name` is true (1), and `is_part_number` is false (0). The `action` field is ignored as per the instructions. 

Please note that the JSON data does not provide any information about the `action` field, so it's not included in the WHERE clause. Also, the `is_brand_name` and `is_part_number` fields are assumed to be boolean, with `true` represented as 1 and `false` as 0.

--------------------

Note saved without feedback. ID: a687b3e8-ff8a-43fd-987a-06c3f110fa22
Input tokens: 177 Output tokens: 191 Total price: 0.02$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [133]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

ghconv = GoogleSheetToSQLiteConverter(
    spreadsheet='Famaga Knowledge Map', 
    credentials_path=r'C:\Users\MGroup\Downloads\langchain-400510-ac38916fe0c8.json')

sheet_data = ghconv.get_sheet_data('Classify parts')

product_inquiries = [
    ProductInquiry(
        intent=item['Intent'],
        sub_intent=item['Sub-intent'],
        is_brand_name=item['Brand name'],
        is_part_number=item['Part Number'],
        action=item['Action']
    )
    for item in sheet_data
]

init_sqlite_db(conn, product_inquiries)


Data was added to db successfully.


In [126]:
cursor.execute(raw_sql)

In [134]:
cursor.execute(raw_sql)
rows = cursor.fetchall()

for row in rows:
    print(row)

('Product Inquiry', 'Availability Check', 1, 0, 'ask_specify_part_number')


In [ ]:
MsgHtmlExtracter.extract_messages_from_raw_html()

In [159]:
messages_html = deals_rep.get_messaging_html(415947)
# resp = client.handle_messages(123,messages_html)

In [160]:
messages = MsgHtmlExtracter.extract_messages_from_raw_html(messages_html)

In [161]:
print('\n\n'.join([f'Message:{m}' if 'for orders under 150 Euro' not in m  else 'Message: Offer' for m in messages[::-1]]))

Message:< o\:*
                  {behavior:url(#default#VML);} w\:*
                  {behavior:url(#default#VML);} .shape
                  {behavior:url(#default#VML);}Hi please be so kind and quote us
                  on:059091816
                    eachRECEIVER:
                    TYPE: LIGHT; MANUF P/N: LR 100TB 58; HOUSING
                    MATERIAL: NICKELPLATED
                    BRASS; HOUSING M12 X 1; CONNECTION 4 PIN M12 PLUG;
                    SENSING RANGE UP TO18
                    METERS; SEALING CLASS AP67____________________________________________________________________________________059092116
                    eachSENSOR:
                    TYPE: REMOTE; RANGE: 24 M; RATING: 10-55 HZ;
                    ELECTRICALCONNECTION:
                    4 PIN; MOUNT: SCREW IN; DIMENSIONS: DIA 12 X LG 58
                    MM; PROBEDIAMETER:
                    12 MM; MANUF P/N: LT 100H TB58 J; OPERATIONAL
                    TEMPERATURE: -25-65 DEGC;
         

In [162]:
client = AgentsAPIClient('https://neon-dev.us')


In [165]:
client.handle_html_messages(deal_id=322, messages_html=messages_html)

Exception: Failed to upload email content: 500 - Internal Server Error